In [73]:
import pandas as pd
import numpy as np

from functools import reduce

from sqlalchemy import create_engine

from datetime import date

import matplotlib.pyplot as plt
import seaborn as sns

# 7 Total Possible Groups of Users

1. [Users that did not have any offers, only transactions](#only_transactions)
2. [Users that received offers](#r)
    1. [Users that viewed offers](#rv)
        1. [Users that completed offers](#rvc)
        2. [Users that did not complete offers, but still had transactions](#rvt)
        3. [Users that did not have transactions](#rvn)
    2. [Users that did not view offers](#rn)
        1. [Users that completed offers](#rnc)
        2. [Users that did not complete offers, but still had transactions](#rnt)
        3. [Users that did not have transactions](#rnn)

Loading data

In [74]:
conn = create_engine('sqlite:///data/raw_starbucks.db')

# offers.to_sql('offers', conn, if_exists='replace', index=False)
# users.to_sql('users', conn, if_exists='replace', index=False)
# interactions.to_sql('interactions', conn, if_exists='replace', index=False)

offers = pd.read_sql('SELECT * FROM offers', conn)

users = pd.read_sql('SELECT * FROM users', conn)
users['user_age_group'] = users['user_age_group'].fillna('Unknown')
users['user_age_group'] = users['user_age_group'].astype(str)
categories = ['Unknown'] + sorted(users['user_age_group'].unique().tolist())[:-1]
users['user_age_group'] = pd.Categorical(users['user_age_group'], categories=categories, ordered=True)

interactions = pd.read_sql('SELECT * FROM interactions', conn)

In [75]:
raw_interactions = pd.read_json('data/transcript.json', orient='records', lines=True)

In [76]:
offer_received = interactions[ interactions['intxn_event_type'] == 'offer received' ].copy().drop(['intxn_amount', 'intxn_reward'], axis=1)
offer_received_cols = [ "received" + cn.removeprefix('intxn') if 'intxn' in cn else cn for cn in offer_received.columns ]
offer_received.columns = offer_received_cols

offer_viewed = interactions[ interactions['intxn_event_type'] == 'offer viewed' ].copy().drop(['intxn_amount', 'intxn_reward'], axis=1)
offer_viewed_cols = [ "viewed" + cn.removeprefix('intxn') if 'intxn' in cn else cn for cn in offer_viewed.columns ]
offer_viewed.columns = offer_viewed_cols

offer_completed = interactions[ interactions['intxn_event_type'] == 'offer completed' ].copy().drop(['intxn_amount'], axis=1)
offer_completed_cols = [ "completed" + cn.removeprefix('intxn') if 'intxn' in cn else cn for cn in offer_completed.columns ]
offer_completed.columns = offer_completed_cols

transactions = interactions[ interactions['intxn_event_type'] == 'transaction' ].copy().drop(['offer_id', 'intxn_reward'], axis=1)
transactions_cols = [ "transaction" + cn.removeprefix('intxn') if 'intxn' in cn else cn for cn in transactions.columns ]
transactions.columns = transactions_cols

<a id='only_transactions'></a>
# Users with only Transactions

In [77]:
transactions_no_promos = pd.merge( transactions, offer_received, on='user_id', how='left' )
transactions_no_promos = transactions_no_promos[ transactions_no_promos['offer_id'].isna() ]

transactions_no_promos

,user_id,transaction_event_type,transaction_time,transaction_amount,received_event_type,received_time,offer_id
4096,eb540099db834cf59001f83a4561aef3,transaction,6,4.74,NaN,NaN,NaN
14148,12ede229379747bd8d74ccdc20097ca3,transaction,18,16.62,NaN,NaN,NaN
124269,c6e579c6821c41d1a7a6a9cf936e91bb,transaction,174,0.65,NaN,NaN,NaN
166666,c6e579c6821c41d1a7a6a9cf936e91bb,transaction,222,1.91,NaN,NaN,NaN
238452,ae8111e7e8cd4b60a8d35c42c1110555,transaction,306,17.51,NaN,NaN,NaN
244267,eb540099db834cf59001f83a4561aef3,transaction,318,5.09,NaN,NaN,NaN
248661,3a4874d8f0ef42b9a1b72294902afea9,transaction,324,22.75,NaN,NaN,NaN
250562,12ede229379747bd8d74ccdc20097ca3,transaction,324,27.60,NaN,NaN,NaN
254212,ae8111e7e8cd4b60a8d35c42c1110555,transaction,330,28.70,NaN,NaN,NaN
294458,3a4874d8f0ef42b9a1b72294902afea9,transaction,378,27.20,NaN,NaN,NaN


<a id='r'></a>
# Users that received offers

In [78]:
offer_received.head()

,user_id,received_event_type,received_time,offer_id
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,0,9b98b8c7a33c4b65b9aebfe6a799e6d9
1,a03223e636434f42ac4c3df47e8bac43,offer received,0,0b1e1539f2cc45b7b9fa7c272da2e1d7
2,e2127556f4f64592b11af22de27a7932,offer received,0,2906b810c7d4411798c6938adc9daaa5
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,0,fafdcd668e3743c1bb461111dcafc2a4
4,68617ca6246f4fbc85e91a2a49552598,offer received,0,4d5c57ea9a6940dd891ad53e9dbe8da0


<a id='rv'></a>
## Users that viewed offers

In [79]:
offer_received_viewed = pd.merge( offer_received, offer_viewed, on=['user_id', 'offer_id'], how='left' )
offer_received_viewed = offer_received_viewed[ ~offer_received_viewed['viewed_time'].isna() ]
# Removing duplicates caused by receiving the same offer multiple times and the join including those received offers before the viewed ones
offer_received_viewed = offer_received_viewed[ offer_received_viewed['received_time'] <= offer_received_viewed['viewed_time'] ]
# Removing duplicates caused by viewing the same offer after receiving it another time
offer_received_viewed = offer_received_viewed.sort_values(['user_id', 'received_time', 'viewed_time']).drop_duplicates(subset=['offer_id', 'user_id', 'received_time'], keep='first')

offer_received_viewed

,user_id,received_event_type,received_time,offer_id,viewed_event_type,viewed_time
19437,0009655768c64bdeb2e877511632db8f,offer received,168,5a8bc65990b245e5a138643cd4eb9837,offer viewed,192.0
35295,0009655768c64bdeb2e877511632db8f,offer received,336,3f207df678b143eea3cee63160fa8bed,offer viewed,372.0
51166,0009655768c64bdeb2e877511632db8f,offer received,408,f19421c1d4aa40978ebb69ca19b0e20d,offer viewed,456.0
66965,0009655768c64bdeb2e877511632db8f,offer received,504,fafdcd668e3743c1bb461111dcafc2a4,offer viewed,540.0
20066,00116118485d4dfda04fdbaba9a87b5c,offer received,168,f19421c1d4aa40978ebb69ca19b0e20d,offer viewed,216.0
...,...,...,...,...,...,...
17694,ffff82501cea40309d5fdd7edcca4a07,offer received,168,0b1e1539f2cc45b7b9fa7c272da2e1d7,offer viewed,174.0
33595,ffff82501cea40309d5fdd7edcca4a07,offer received,336,2906b810c7d4411798c6938adc9daaa5,offer viewed,354.0
49455,ffff82501cea40309d5fdd7edcca4a07,offer received,408,2906b810c7d4411798c6938adc9daaa5,offer viewed,414.0
65289,ffff82501cea40309d5fdd7edcca4a07,offer received,504,9b98b8c7a33c4b65b9aebfe6a799e6d9,offer viewed,534.0


<a id='rvc'></a>
### Users that completed the offer

In [80]:
offer_received_viewed_completed = pd.merge( offer_received_viewed, offer_completed, on=['user_id', 'offer_id'], how='left' )
offer_received_viewed_completed = offer_received_viewed_completed[ ~offer_received_viewed_completed['completed_time'].isna() ]
# Removing duplicates caused by completing the same offer before receiving it another time
offer_received_viewed_completed = offer_received_viewed_completed[ offer_received_viewed_completed['completed_time'] >= offer_received_viewed_completed['received_time'] ]
# Removing duplicates caused by completing the same offer after viewing it another time
offer_received_viewed_completed = offer_received_viewed_completed[ offer_received_viewed_completed['completed_time'] >= offer_received_viewed_completed['viewed_time'] ]
# Removing duplicates caused by viewing the same offer after receiving it another time
offer_received_viewed_completed = offer_received_viewed_completed.sort_values(['user_id', 'received_time', 'viewed_time', 'completed_time']).drop_duplicates(subset=['offer_id', 'user_id', 'received_time', 'viewed_time'], keep='first')

offer_received_viewed_completed

,user_id,received_event_type,received_time,offer_id,viewed_event_type,viewed_time,completed_event_type,completed_time,completed_reward
7,0011e0d4e6b944f998e987f904e8c1e5,offer received,168,2298d6c36e964ae4a3e7e9706d1fb8c2,offer viewed,186.0,offer completed,252.0,3.0
9,0011e0d4e6b944f998e987f904e8c1e5,offer received,408,0b1e1539f2cc45b7b9fa7c272da2e1d7,offer viewed,432.0,offer completed,576.0,5.0
10,0011e0d4e6b944f998e987f904e8c1e5,offer received,504,9b98b8c7a33c4b65b9aebfe6a799e6d9,offer viewed,516.0,offer completed,576.0,5.0
11,0020c2b971eb4e9188eac86d93036a77,offer received,0,fafdcd668e3743c1bb461111dcafc2a4,offer viewed,12.0,offer completed,54.0,2.0
13,0020c2b971eb4e9188eac86d93036a77,offer received,408,4d5c57ea9a6940dd891ad53e9dbe8da0,offer viewed,426.0,offer completed,510.0,10.0
...,...,...,...,...,...,...,...,...,...
67882,fffad4f4828548d1b5583907f2e9906b,offer received,408,f19421c1d4aa40978ebb69ca19b0e20d,offer viewed,510.0,offer completed,516.0,5.0
67884,ffff82501cea40309d5fdd7edcca4a07,offer received,0,fafdcd668e3743c1bb461111dcafc2a4,offer viewed,6.0,offer completed,60.0,2.0
67885,ffff82501cea40309d5fdd7edcca4a07,offer received,168,0b1e1539f2cc45b7b9fa7c272da2e1d7,offer viewed,174.0,offer completed,198.0,5.0
67886,ffff82501cea40309d5fdd7edcca4a07,offer received,336,2906b810c7d4411798c6938adc9daaa5,offer viewed,354.0,offer completed,384.0,2.0


In [53]:
offer_received_viewed_completed.sort_values(['user_id', 'received_time', 'viewed_time', 'completed_time'])#.drop_duplicates(subset=['offer_id', 'user_id', 'received_time'], keep='first')

,user_id,received_event_type,received_time,offer_id,viewed_event_type,viewed_time,completed_event_type,completed_time,completed_reward
7,0011e0d4e6b944f998e987f904e8c1e5,offer received,168,2298d6c36e964ae4a3e7e9706d1fb8c2,offer viewed,186.0,offer completed,252.0,3.0
9,0011e0d4e6b944f998e987f904e8c1e5,offer received,408,0b1e1539f2cc45b7b9fa7c272da2e1d7,offer viewed,432.0,offer completed,576.0,5.0
10,0011e0d4e6b944f998e987f904e8c1e5,offer received,504,9b98b8c7a33c4b65b9aebfe6a799e6d9,offer viewed,516.0,offer completed,576.0,5.0
11,0020c2b971eb4e9188eac86d93036a77,offer received,0,fafdcd668e3743c1bb461111dcafc2a4,offer viewed,12.0,offer completed,54.0,2.0
12,0020c2b971eb4e9188eac86d93036a77,offer received,0,fafdcd668e3743c1bb461111dcafc2a4,offer viewed,12.0,offer completed,510.0,2.0
...,...,...,...,...,...,...,...,...,...
67886,ffff82501cea40309d5fdd7edcca4a07,offer received,336,2906b810c7d4411798c6938adc9daaa5,offer viewed,354.0,offer completed,384.0,2.0
67887,ffff82501cea40309d5fdd7edcca4a07,offer received,336,2906b810c7d4411798c6938adc9daaa5,offer viewed,354.0,offer completed,414.0,2.0
67888,ffff82501cea40309d5fdd7edcca4a07,offer received,336,2906b810c7d4411798c6938adc9daaa5,offer viewed,354.0,offer completed,576.0,2.0
67890,ffff82501cea40309d5fdd7edcca4a07,offer received,408,2906b810c7d4411798c6938adc9daaa5,offer viewed,414.0,offer completed,414.0,2.0


In [57]:
offer_received_viewed_completed[ offer_received_viewed_completed['user_id'] == '0020c2b971eb4e9188eac86d93036a77' ].sort_values(['user_id', 'received_time', 'viewed_time', 'completed_time']).drop_duplicates(subset=['offer_id', 'user_id', 'received_time', 'viewed_time'], keep='first')

,user_id,received_event_type,received_time,offer_id,viewed_event_type,viewed_time,completed_event_type,completed_time,completed_reward
11,0020c2b971eb4e9188eac86d93036a77,offer received,0,fafdcd668e3743c1bb461111dcafc2a4,offer viewed,12.0,offer completed,54.0,2.0
13,0020c2b971eb4e9188eac86d93036a77,offer received,408,4d5c57ea9a6940dd891ad53e9dbe8da0,offer viewed,426.0,offer completed,510.0,10.0


In [71]:
test_df = raw_interactions[ (raw_interactions['person'] == '0020c2b971eb4e9188eac86d93036a77') ].sort_values('time')
test_df['offer id'] = test_df['value'].apply(lambda x: x.get('offer id') if isinstance(x, dict) and 'offer id' in x else None)
test_df['offer_id'] = test_df['value'].apply(lambda x: x.get('offer_id') if isinstance(x, dict) and 'offer_id' in x else None)
test_df = test_df[( test_df['offer id'] == 'fafdcd668e3743c1bb461111dcafc2a4') | (test_df['offer_id'] == 'fafdcd668e3743c1bb461111dcafc2a4') ]
test_df.drop(['offer_id', 'offer id'], axis=1, inplace=True)

test_df

,person,event,value,time
1889,0020c2b971eb4e9188eac86d93036a77,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
18431,0020c2b971eb4e9188eac86d93036a77,offer viewed,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},12
31327,0020c2b971eb4e9188eac86d93036a77,offer completed,{'offer_id': 'fafdcd668e3743c1bb461111dcafc2a4...,54
112698,0020c2b971eb4e9188eac86d93036a77,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},336
218936,0020c2b971eb4e9188eac86d93036a77,offer completed,{'offer_id': 'fafdcd668e3743c1bb461111dcafc2a4...,510


<a id='rvt'></a>
### Users that did not complete the offer

In [104]:
offer_received_viewed_nocomp = pd.merge( offer_received_viewed, offer_completed, on=['user_id', 'offer_id'], how='left' )
offer_received_viewed_nocomp = offer_received_viewed_nocomp[ offer_received_viewed_nocomp['completed_time'].isna() ]
offer_received_viewed_nocomp = pd.merge( offer_received_viewed_nocomp, transactions, on=['user_id'], how='left' )
offer_received_viewed_nocomp = offer_received_viewed_nocomp[ offer_received_viewed_nocomp['transaction_time'] > offer_received_viewed_nocomp['received_time'] ]
offer_received_viewed_nocomp = offer_received_viewed_nocomp[ offer_received_viewed_nocomp['transaction_time'] > offer_received_viewed_nocomp['viewed_time'] ]
offer_received_viewed_nocomp_agg = offer_received_viewed_nocomp.groupby(['user_id', 'offer_id', 'received_event_type', 'viewed_event_type']).agg(
    offer_received_time=('received_time', 'first'),
    offer_viewed_time=('viewed_time', 'first'),
    transaction_time=('transaction_time', 'last'),
    transaction_count=('transaction_time', 'count'),
    transaction_sum=('transaction_amount', 'sum')
).reset_index()

offer_received_viewed_nocomp_agg

,user_id,offer_id,received_event_type,viewed_event_type,offer_received_time,offer_viewed_time,transaction_time,transaction_count,transaction_sum
0,0009655768c64bdeb2e877511632db8f,3f207df678b143eea3cee63160fa8bed,offer received,offer viewed,336,372.0,696.0,7,105.44
1,0009655768c64bdeb2e877511632db8f,5a8bc65990b245e5a138643cd4eb9837,offer received,offer viewed,168,192.0,696.0,8,127.60
2,00116118485d4dfda04fdbaba9a87b5c,f19421c1d4aa40978ebb69ca19b0e20d,offer received,offer viewed,168,216.0,474.0,3,4.09
3,0011e0d4e6b944f998e987f904e8c1e5,3f207df678b143eea3cee63160fa8bed,offer received,offer viewed,0,6.0,654.0,5,79.46
4,0011e0d4e6b944f998e987f904e8c1e5,5a8bc65990b245e5a138643cd4eb9837,offer received,offer viewed,336,354.0,654.0,3,54.04
...,...,...,...,...,...,...,...,...,...
21112,fff3ba4757bd42088c044ca26d73817a,5a8bc65990b245e5a138643cd4eb9837,offer received,offer viewed,336,540.0,552.0,2,52.18
21113,fff7576017104bcc8677a8d63322b5e1,4d5c57ea9a6940dd891ad53e9dbe8da0,offer received,offer viewed,168,180.0,696.0,5,24.39
21114,fff7576017104bcc8677a8d63322b5e1,ae264e3637204a6fb9bb56bc8210ddfd,offer received,offer viewed,408,414.0,696.0,3,13.89
21115,fff8957ea8b240a6b5e634b6ee8eafcf,fafdcd668e3743c1bb461111dcafc2a4,offer received,offer viewed,408,432.0,576.0,3,8.09


In [105]:
test_offer_ids = offer_received_viewed_nocomp['offer_id'].unique().tolist()

offers[ offers['offer_id'].isin(test_offer_ids) ].sort_values('offer_id')

,offer_id,offer_notification_channel,offer_type,offer_spend_minimum,offer_reward,offer_duration
14,0b1e1539f2cc45b7b9fa7c272da2e1d7,email,discount,20,5,240
13,0b1e1539f2cc45b7b9fa7c272da2e1d7,web,discount,20,5,240
16,2298d6c36e964ae4a3e7e9706d1fb8c2,email,discount,7,3,168
18,2298d6c36e964ae4a3e7e9706d1fb8c2,social,discount,7,3,168
17,2298d6c36e964ae4a3e7e9706d1fb8c2,mobile,discount,7,3,168
15,2298d6c36e964ae4a3e7e9706d1fb8c2,web,discount,7,3,168
30,2906b810c7d4411798c6938adc9daaa5,web,discount,10,2,168
31,2906b810c7d4411798c6938adc9daaa5,email,discount,10,2,168
32,2906b810c7d4411798c6938adc9daaa5,mobile,discount,10,2,168
9,3f207df678b143eea3cee63160fa8bed,mobile,informational,0,0,96


In [108]:
offer_received_viewed_nocomp_agg[ (offer_received_viewed_nocomp_agg['offer_id'] == "0b1e1539f2cc45b7b9fa7c272da2e1d7") & (offer_received_viewed_nocomp_agg['user_id'] == '009d10c2c38a4fd795a7bc6ddbbddb78') ]

,user_id,offer_id,received_event_type,viewed_event_type,offer_received_time,offer_viewed_time,transaction_time,transaction_count,transaction_sum
29,009d10c2c38a4fd795a7bc6ddbbddb78,0b1e1539f2cc45b7b9fa7c272da2e1d7,offer received,offer viewed,168,228.0,684.0,3,3.3


In [94]:
test_df = raw_interactions[ (raw_interactions['person'] == '0009655768c64bdeb2e877511632db8f') ].sort_values('time')
test_df['offer id'] = test_df['value'].apply(lambda x: x.get('offer id') if isinstance(x, dict) and 'offer id' in x else None)
test_df['offer_id'] = test_df['value'].apply(lambda x: x.get('offer_id') if isinstance(x, dict) and 'offer_id' in x else None)
# test_df = test_df[( test_df['offer id'] == '3f207df678b143eea3cee63160fa8bed') | (test_df['offer_id'] == '3f207df678b143eea3cee63160fa8bed') ]
# test_df.drop(['offer_id', 'offer id'], axis=1, inplace=True)

test_df

,person,event,value,time,offer id,offer_id
55972,0009655768c64bdeb2e877511632db8f,offer received,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'},168,5a8bc65990b245e5a138643cd4eb9837,None
77705,0009655768c64bdeb2e877511632db8f,offer viewed,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'},192,5a8bc65990b245e5a138643cd4eb9837,None
89291,0009655768c64bdeb2e877511632db8f,transaction,{'amount': 22.16},228,None,None
113605,0009655768c64bdeb2e877511632db8f,offer received,{'offer id': '3f207df678b143eea3cee63160fa8bed'},336,3f207df678b143eea3cee63160fa8bed,None
139992,0009655768c64bdeb2e877511632db8f,offer viewed,{'offer id': '3f207df678b143eea3cee63160fa8bed'},372,3f207df678b143eea3cee63160fa8bed,None
153401,0009655768c64bdeb2e877511632db8f,offer received,{'offer id': 'f19421c1d4aa40978ebb69ca19b0e20d'},408,f19421c1d4aa40978ebb69ca19b0e20d,None
168412,0009655768c64bdeb2e877511632db8f,transaction,{'amount': 8.57},414,None,None
168413,0009655768c64bdeb2e877511632db8f,offer completed,{'offer_id': 'f19421c1d4aa40978ebb69ca19b0e20d...,414,None,f19421c1d4aa40978ebb69ca19b0e20d
187554,0009655768c64bdeb2e877511632db8f,offer viewed,{'offer id': 'f19421c1d4aa40978ebb69ca19b0e20d'},456,f19421c1d4aa40978ebb69ca19b0e20d,None
204340,0009655768c64bdeb2e877511632db8f,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},504,fafdcd668e3743c1bb461111dcafc2a4,None


<a id='rvn'></a>
### Users that did not have transactions

In [25]:
offer_received_viewed_notrans = pd.merge( offer_received_viewed, transactions, on=['user_id'], how='left' )
offer_received_viewed_notrans = offer_received_viewed_notrans[ offer_received_viewed_notrans['transaction_time'].isna() ]

offer_received_viewed_notrans

,user_id,received_event_type,received_time,offer_id,viewed_event_type,viewed_time,transaction_event_type,transaction_time,transaction_amount
16,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,0,fafdcd668e3743c1bb461111dcafc2a4,offer viewed,12.0,NaN,NaN,NaN
17,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,0,fafdcd668e3743c1bb461111dcafc2a4,offer viewed,522.0,NaN,NaN,NaN
623,13b7b2eccf664a329d83d2a238a9a11d,offer received,0,f19421c1d4aa40978ebb69ca19b0e20d,offer viewed,18.0,NaN,NaN,NaN
624,13b7b2eccf664a329d83d2a238a9a11d,offer received,0,f19421c1d4aa40978ebb69ca19b0e20d,offer viewed,414.0,NaN,NaN,NaN
1395,accec36a10704e1fbaeb9b28b8bc5da8,offer received,0,fafdcd668e3743c1bb461111dcafc2a4,offer viewed,66.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
684342,0f4eeccda01547bdaa30113a09725a2b,offer received,576,3f207df678b143eea3cee63160fa8bed,offer viewed,618.0,NaN,NaN,NaN
684741,4c9c818067914caba08ca3cebadab165,offer received,576,9b98b8c7a33c4b65b9aebfe6a799e6d9,offer viewed,6.0,NaN,NaN,NaN
684742,4c9c818067914caba08ca3cebadab165,offer received,576,9b98b8c7a33c4b65b9aebfe6a799e6d9,offer viewed,576.0,NaN,NaN,NaN
685154,1f5e62e325d047f4833d1b7aed437f31,offer received,576,fafdcd668e3743c1bb461111dcafc2a4,offer viewed,588.0,NaN,NaN,NaN


<a id='rn'></a>
## Users that did not view offers

In [ ]:
offer_not_viewed = pd.merge( offer_received, offer_viewed, on=['user_id', 'offer_id'], how='left' )
offer_not_viewed = offer_not_viewed[ offer_not_viewed['viewed_time'].isna() ]

display(offer_not_viewed)

In [ ]:
user_id = '868317b9be554cb18e50bc68484749a2'
offer_id = '2906b810c7d4411798c6938adc9daaa5'

display( interactions[ (interactions['user_id'] == user_id) & (interactions['offer_id']==offer_id) ].sort_values( by=['intxn_time'] ) )
display( transactions[ transactions['user_id'] == user_id ] )
display( offers[ offers['offer_id'] == offer_id ] )

<a id='rnc'></a>
### Users that completed the offer

<a id='rnv'></a>
### Users that did not complete the offer

In [ ]:
offer_not_completed = pd.merge( offer_viewed, offer_completed, on=['user_id', 'offer_id'], how='left' )
offer_not_completed = offer_not_completed[ offer_not_completed['completed_event_type'].isna() ]

display(offer_not_completed)

In [ ]:
user_id = 'd1ede868e29245ea91818a903fec04c6'
offer_id = '5a8bc65990b245e5a138643cd4eb9837'

display( interactions[ (interactions['user_id'] == user_id) & (interactions['offer_id']==offer_id) ].sort_values( by=['intxn_time'] ) )
display( transactions[ transactions['user_id'] == user_id ] )
display( offers[ offers['offer_id'] == offer_id ] )

<a id='rnn'></a>
### Users that did not have transactions